# Object Tracking with Siamese Networks

In this session we will train a siamese network to optimize the euclidean distance between positive and negative image pairs. This will serve as a toy example of a tracking scenario, in which we need to match a set of detected objects in two consecutive frames. We will use MNIST dataset to train this network.

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

Here we define the loss function that we want to use, which uses the euclidean distance as metric. The loss is defined so that it minimizes the distance between positive pairs and maximizes it for negative pairs, with a certain margin.

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

We need to define the pairs of positives and negatives for training. A positive pair will be compose of two samples that belong to the same category. The function ```create_pairs``` will do this for us:

In [ ]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    
    # number of samples per category
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            # z1 are positive pairs
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            # add the samples of z1 and x2 indices to pairs
            pairs += [[x[z1], x[z2]]]
            # select random sample from another category
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

Here we creat a simple architecture. This will be shared between the two samples in the pairs.

In [ ]:
def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq

Since we defined a network with fully connected layers, we need to flatten the images as we did in the exercices from the first day.

In [ ]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
input_dim = 784

Create positive and negative pairs for training and testing splits:

In [ ]:
nb_epoch = 20

# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(10)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(10)]
te_pairs, te_y = create_pairs(X_test, digit_indices)

Now we build the siamese network by reusing the base network we previously defined:

In [ ]:
# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

In order to use the loss we defined above, we need to create a layer that computes the euclidean distance between the output of the two network branches. The euclidean distance will be the output of the network, which is the input to the loss function.

In [ ]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
model = Model(input=[input_a, input_b], output=distance)

We are now ready to train:

In [ ]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          batch_size=128,
          nb_epoch=nb_epoch,
          verbose=2)

Now we have our network trained and ready to be used on a toy tracking example. We will start by choosing some MNIST samples as if they were detected objects. 

We write a function to plot the samples we will select as detections:

In [ ]:
import matplotlib.pyplot as plt  
%matplotlib inline

def plot_samples(samples):

    f, axarr = plt.subplots(1,samples.shape[0])
    for i in range(samples.shape[0]):
        
        im = np.reshape(samples[i],(28,28))
        axarr[i].imshow(im)
        axarr[i].axis('off')
    

    plt.show()

Now we will pick a different sample from each of the 10 different classes. These will be the detections on the two different frames, and we need to find a network to match them:

In [ ]:
frame_1_idxs = []
frame_2_idxs = []

for i in range(len(digit_indices)):
    frame_1_idxs.append(digit_indices[i][0])
    frame_2_idxs.append(digit_indices[i][1])
frame_1_samples = X_test[frame_1_idxs,:]
frame_2_samples = X_test[frame_2_idxs,:]

plot_samples(frame_1_samples)
plot_samples(frame_2_samples)

Now let's create the extractor that we will use to extract the features from our network for all these samples:

In [ ]:
model.layers

We need to forward our samples through the *Sequential* network within our model and get its output as the feature representation of our samples.

In [ ]:
base_net = model.layers[2]

**Exercise:** Use ```base_net``` to extract features from detections in both frames (using the ```predict``` method):

In [ ]:
feats_frame_1 = ...
feats_frame_2 = ...

**Exercise:** Given the two sets of features we just extracted, write the code to find the best match for each of the detections from frame 1 in frame 2. Check if the results are as you expected. Hint: you can use the function [euclidean_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html) in scikit learn to quickly compute distances between feature vectors.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

The idea here is that after computing the distances, we can see that minimum values are found in the positions where matching samples are found in the other set of detections.